In [11]:
import pandas as pd

# Get credentials
with open('/home/andreas/Dropbox/Projects/scoring-probability/credentials', 'r') as credentials_file:
    URI = credentials_file.read()
    
# Get query
with open('player-scored-query.sql', 'r') as query_file:
    query = query_file.read()
    
# Pandas
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [12]:
# Get the data
result = pd.read_sql(query, URI)

In [13]:
result.head()

,fixture_uid,team_uid,is_home,player_uid,kick_off,gameweek,season,position,played,scored,appearance_number,matches_scored,match_number,team_uid,is_home,position,num_games_scored,num_matches,matches_scored_until_now,appearances_until_now,matches_until_now,chance_score_given_played,chance_score
0,c234ba3c78a51d023833c83b88a7fa5d,9d292673b9c1ebe339be9d36032d5e2e,1,015ab60ee2fcf75b0326823cf47fc32b,2019-08-10 12:30:00,1,1920,gk,0,0,0,0,1,9d292673b9c1ebe339be9d36032d5e2e,1,gk,0,15,0,15,15,0,0
1,d7527be5828734075d0caa9194c69437,9d292673b9c1ebe339be9d36032d5e2e,0,015ab60ee2fcf75b0326823cf47fc32b,2019-08-17 15:00:00,2,1920,gk,0,0,0,0,2,9d292673b9c1ebe339be9d36032d5e2e,0,gk,0,16,0,16,17,0,0
2,282a535c1a57b5aa490b45927d41c57a,9d292673b9c1ebe339be9d36032d5e2e,0,015ab60ee2fcf75b0326823cf47fc32b,2019-08-24 15:00:00,3,1920,gk,0,0,0,0,3,9d292673b9c1ebe339be9d36032d5e2e,0,gk,0,16,0,16,18,0,0
3,51c192c4d46d8e91d3068f0fd56d3e75,9d292673b9c1ebe339be9d36032d5e2e,1,015ab60ee2fcf75b0326823cf47fc32b,2019-08-31 15:00:00,4,1920,gk,0,0,0,0,4,9d292673b9c1ebe339be9d36032d5e2e,1,gk,0,15,0,15,18,0,0
4,754a58406ca76df91d773087f45f2ccd,9d292673b9c1ebe339be9d36032d5e2e,0,015ab60ee2fcf75b0326823cf47fc32b,2019-09-16 20:00:00,5,1920,gk,0,0,0,0,5,9d292673b9c1ebe339be9d36032d5e2e,0,gk,0,16,0,16,20,0,0


In [14]:
# Get some info on the player
player_query = """
select uid as player_uid, name from data_players
"""

player_info = pd.read_sql(player_query, URI)

In [15]:
result_w_info = result.merge(player_info)

In [16]:
result_w_info

,fixture_uid,team_uid,is_home,player_uid,kick_off,gameweek,season,position,played,scored,appearance_number,matches_scored,match_number,team_uid,is_home,position,num_games_scored,num_matches,matches_scored_until_now,appearances_until_now,matches_until_now,chance_score_given_played,chance_score,name
0,c234ba3c78a51d023833c83b88a7fa5d,9d292673b9c1ebe339be9d36032d5e2e,1,015ab60ee2fcf75b0326823cf47fc32b,2019-08-10 12:30:00,1,1920,gk,0,0,0,0,1,9d292673b9c1ebe339be9d36032d5e2e,1,gk,0,15,0,15,15,0,0,Roberto
1,d7527be5828734075d0caa9194c69437,9d292673b9c1ebe339be9d36032d5e2e,0,015ab60ee2fcf75b0326823cf47fc32b,2019-08-17 15:00:00,2,1920,gk,0,0,0,0,2,9d292673b9c1ebe339be9d36032d5e2e,0,gk,0,16,0,16,17,0,0,Roberto
2,282a535c1a57b5aa490b45927d41c57a,9d292673b9c1ebe339be9d36032d5e2e,0,015ab60ee2fcf75b0326823cf47fc32b,2019-08-24 15:00:00,3,1920,gk,0,0,0,0,3,9d292673b9c1ebe339be9d36032d5e2e,0,gk,0,16,0,16,18,0,0,Roberto
3,51c192c4d46d8e91d3068f0fd56d3e75,9d292673b9c1ebe339be9d36032d5e2e,1,015ab60ee2fcf75b0326823cf47fc32b,2019-08-31 15:00:00,4,1920,gk,0,0,0,0,4,9d292673b9c1ebe339be9d36032d5e2e,1,gk,0,15,0,15,18,0,0,Roberto
4,754a58406ca76df91d773087f45f2ccd,9d292673b9c1ebe339be9d36032d5e2e,0,015ab60ee2fcf75b0326823cf47fc32b,2019-09-16 20:00:00,5,1920,gk,0,0,0,0,5,9d292673b9c1ebe339be9d36032d5e2e,0,gk,0,16,0,16,20,0,0,Roberto
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16039,04bba9062e80633cad2fb51a837bfe9c,e1d38e23258af8a54310a6f5d8e2e763,0,ff3e766887e6f91d05cba283f93b36a1,2020-02-22 12:30:00,27,1920,centre-back,1,0,18,1,27,e1d38e23258af8a54310a6f5d8e2e763,0,centre-back,2,15,3,32,41,9,7,Jan Vertonghen
16040,8a5ec827534a66b9680101316d8b7c62,e1d38e23258af8a54310a6f5d8e2e763,1,ff3e766887e6f91d05cba283f93b36a1,2020-03-01 14:00:00,28,1920,centre-back,0,0,18,1,28,e1d38e23258af8a54310a6f5d8e2e763,1,centre-back,0,16,1,34,43,2,2,Jan Vertonghen
16041,61bc134812782a24a87ec73d2b885be4,e1d38e23258af8a54310a6f5d8e2e763,0,ff3e766887e6f91d05cba283f93b36a1,2020-03-07 17:30:00,29,1920,centre-back,0,0,18,1,29,e1d38e23258af8a54310a6f5d8e2e763,0,centre-back,2,15,3,33,43,9,6,Jan Vertonghen
16042,4e9f69192069db2c05afb8ea2cb68d39,e1d38e23258af8a54310a6f5d8e2e763,1,ff3e766887e6f91d05cba283f93b36a1,2020-03-15 16:30:00,30,1920,centre-back,0,0,18,1,30,e1d38e23258af8a54310a6f5d8e2e763,1,centre-back,0,16,1,34,45,2,2,Jan Vertonghen
